![The Chicago Maroon](https://chicagomaroon.com/wp-content/uploads/2022/03/Screen-Shot-2022-03-16-at-2.14.23-PM.png)

---

## Shuttle Headways & Passenger Loads Between January 1st and April 8th 2025

In [22]:
from dotenv import load_dotenv
load_dotenv("./.env")
import requests
import os
import pandas as pd
import io
from datetime import datetime, timezone

In [23]:
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://uchicagoshuttles.com"
HEADERS = {
        "key":API_KEY,
        "User-Agent": "curl",
    }

In [28]:
import pytz
from datetime import datetime

# Helper functions getApiData() and utcToCentral() adapted from https://andreithuler.com/UChicagoShuttles/api/examples/visualizations.ipynb

def getApiData(url, key):
    """
    Makes the API request and returns a Pandas dataframe
    """
    headers = {
        "key":API_KEY,
        "User-Agent": "curl",
    }
    
    response = requests.request("GET", BASE_URL+url, headers=headers)
    if response.status_code == 200:
        return(pd.read_csv(io.StringIO(response.text)))
    else:
        raise ValueError(f"ERROR making the API request: {response.text}")

def utcToCentral(datetime_utc):
    """
    Convert naive UTC datetime objects into naive US/Central datetime objects
    """
    datetime_utc = pytz.utc.localize(datetime_utc)
    datetime_central = datetime_utc.astimezone(pytz.timezone('US/Central'))
    datetime_central = datetime_central.replace(tzinfo=None)
    return datetime_central

def utcCurrent():
    """
    Return the current time in UTC.
    """
    return datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

def centralToUtc(datetime_central_str):
    """
    Convert a Chicago (US/Central, naive) datetime string in the format "YYYY-MM-DD HH:MM"
    to its naive UTC datetime string.
    """
    
    central = pytz.timezone('US/Central')
    dt_naive = datetime.strptime(datetime_central_str, "%Y-%m-%d %H:%M:%S")
    dt_central = central.localize(dt_naive)
    dt_utc = dt_central.astimezone(pytz.utc)
    return dt_utc.strftime("%Y-%m-%d %H:%M:%S")

In [29]:
url = f"/api/getStops?start={centralToUtc('2025-01-01 00:00:00')}&end={utcCurrent()}"
df_allstops = getApiData(url, API_KEY)

In [34]:
df_allstops['arrivalTime'] = pd.to_datetime(df_allstops['arrivalTime'])
df_allstops['year'] = df_allstops['arrivalTime'].dt.year
df_allstops['month'] = df_allstops['arrivalTime'].dt.month
df_allstops['date'] = df_allstops['arrivalTime'].dt.day
df_allstops['hour'] = df_allstops['arrivalTime'].dt.hour
df_allstops['dayOfWeek'] = df_allstops['arrivalTime'].dt.dayofweek + 1 # (Monday -> 1)

df_allstops.head()

,id,routeName,stopName,passengerLoad,stopDurationSeconds,arrivalTime,departureTime,stopId,routeId,busId,nextStopId,year,month,date,hour,dayOfWeek
0,619471,Apostolic/Drexel,Apostolic Lot,0,8,2025-01-01 05:59:58,2025-01-01 06:00:06,8614,38730,4315,133007,2025,1,1,5,3
1,619472,North,Reynolds Club,0,451,2025-01-01 05:52:45,2025-01-01 06:00:17,8579,38734,4314,8633,2025,1,1,5,3
2,619473,Apostolic/Drexel,Kenwood & 63rd,0,7,2025-01-01 06:00:18,2025-01-01 06:00:26,133007,38730,4315,8612,2025,1,1,6,3
3,619474,Central,Reynolds Club,0,454,2025-01-01 05:52:56,2025-01-01 06:00:31,8579,38737,4317,141882,2025,1,1,5,3
4,619475,North,Regenstein Library (N),0,19,2025-01-01 06:00:25,2025-01-01 06:00:44,8633,38734,4314,8634,2025,1,1,6,3


## Ridership by stop

In [37]:
# Partially adapted from https://andreithuler.com/UChicagoShuttles/api/examples/visualizations.ipynb

df_total_ridership_per_route = df_allstops.groupby(["stopId", "stopName", "dayOfWeek"])[["passengerLoad"]].sum().reset_index()
df_total_ridership_per_route["passengerLoad"] = df_total_ridership_per_route["passengerLoad"].astype(int)
df_total_ridership_per_route.drop_duplicates(inplace = True)

df_total_ridership_per_route

,stopId,stopName,dayOfWeek,passengerLoad
0,8578,Regents Park Apartments,1,34
1,8578,Regents Park Apartments,2,39
2,8578,Regents Park Apartments,3,53
3,8578,Regents Park Apartments,4,60
4,8578,Regents Park Apartments,5,60
...,...,...,...,...
576,161849,Woodlawn Ave & 61st St,3,5
577,161849,Woodlawn Ave & 61st St,4,11
578,161849,Woodlawn Ave & 61st St,5,5
579,161849,Woodlawn Ave & 61st St,6,4
